In [1]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 28.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import pennylane as qml
from pennylane import numpy as qnp


In [3]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Lambda(lambda x: x.view(-1))])

train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)



100%|██████████| 9.91M/9.91M [00:00<00:00, 229MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 17.4MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 34.8MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 3.16MB/s]


In [4]:
num_qubits = 8
dev = qml.device("default.qubit", wires=num_qubits)

def prepare_quantum_state(image, weights):
    image = qnp.array(image, dtype=np.float32)
    weights = qnp.array(weights, dtype=np.float32)
    for i in range(num_qubits):
        angle = image[i] % (2 * np.pi)
        qml.RY(angle, wires=i)

    for i in range(0, len(weights), 2):
        qml.RZ(weights[i], wires=i % num_qubits)

@qml.qnode(dev)
def swap_test(image1, image2, weights):
    prepare_quantum_state(image1, weights)
    prepare_quantum_state(image2, weights)

    qml.Hadamard(wires=0)
    qml.CSWAP(wires=[0, 1, 2])
    return qml.expval(qml.PauliZ(0))


In [5]:
def contrastive_loss(fidelity, label1, label2, margin=0.5):
    if label1 == label2:
        return (1 - fidelity)**2
    else:
        return qnp.array(max(0, fidelity)**2, dtype=float)

In [6]:
def train_qnn_with_negative_pairs(model, train_loader, num_epochs=10, learning_rate=0.01):
    opt = qml.AdamOptimizer(learning_rate)
    weights = qnp.random.uniform(-np.pi, np.pi, num_qubits, requires_grad=True)  # Use qml.numpy for weights

    for epoch in range(num_epochs):
        total_loss = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            idx1 = np.random.randint(0, data.size(0))
            idx2 = np.random.randint(0, data.size(0))

            image1 = data[idx1].squeeze(0).numpy()
            image2 = data[idx2].squeeze(0).numpy()

            label1 = target[idx1].item()
            label2 = target[idx2].item()

            image1 = np.array(image1, dtype=np.float32)
            image2 = np.array(image2, dtype=np.float32)

            # loss function
            def loss_fn(weights):
                fidelity = swap_test(image1, image2, weights)
                loss = contrastive_loss(fidelity, label1, label2)
                return loss

            weights = opt.step(loss_fn, weights)
            total_loss += loss_fn(weights)

        print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}")
    return weights



In [7]:
def evaluate_qnn(model, test_loader, weights):
    correct = 0
    total = 0
    for data, target in test_loader:
        image1 = data[0].squeeze(0).numpy()
        image2 = data[1].squeeze(0).numpy()

        label1 = target[0].item()
        label2 = target[1].item()

        image1 = np.array(image1, dtype=np.float32)
        image2 = np.array(image2, dtype=np.float32)

        fidelity = swap_test(image1, image2, weights)

        prediction = 1 if fidelity > 0.5 else 0
        if prediction == (label1 == label2):
            correct += 1
        total += 1

    accuracy = correct / total
    print(f"Accuracy on the test set: {accuracy * 100:.2f}%")


In [8]:
weights = train_qnn_with_negative_pairs(swap_test, train_loader, num_epochs=10, learning_rate=0.01)

# Evaluate
evaluate_qnn(swap_test, test_loader, weights)


/usr/local/lib/python3.11/dist-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


Epoch 1, Loss: 0.10980810234541578
Epoch 2, Loss: 0.10767590618336886
Epoch 3, Loss: 0.1162046908315565
Epoch 4, Loss: 0.11940298507462686
Epoch 5, Loss: 0.11300639658848614
Epoch 6, Loss: 0.1140724946695096
Epoch 7, Loss: 0.09914712153518124
Epoch 8, Loss: 0.11833688699360341
Epoch 9, Loss: 0.11940298507462686
Epoch 10, Loss: 0.1257995735607676
Accuracy on the test set: 92.36%
